In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
import string
from wordcloud import WordCloud
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV,StratifiedKFold,RandomizedSearchCV

In [ ]:
train = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')
test = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train.info()

In [ ]:
train.shape

In [ ]:
train.target.unique()

In [ ]:
train.location.nunique()

In [ ]:
train.keyword.nunique()

In [ ]:
test.info()

In [ ]:
test.shape

In [ ]:
test.keyword.nunique()

In [ ]:
test.location.nunique()

In [ ]:
train.isna().sum()

In [ ]:
test.isna().sum()

In [ ]:
train.target.value_counts(normalize=True)

#### Observation - Since, value for target variable is almost equal in number, we can apply machine learning algorithm on it.

In [ ]:
plt.figure(figsize=(8,4))
sns.barplot(train.target.value_counts().index, train.target.value_counts())

In [ ]:
#train.location.value_counts().nlargest(10)
plt.figure(figsize=(10,4))
sns.barplot(train.location.value_counts().nlargest(10),train.location.value_counts().nlargest(10).index, orient='h')

In [ ]:
#https://stackoverflow.com/questions/26540035/rotate-label-text-in-seaborn-factorplot

#train.keyword.value_counts().nlargest(20)
plt.figure(figsize=(15,6))
sns.barplot(train.keyword.value_counts().nlargest(20).index,train.keyword.value_counts().nlargest(20))
plt.xticks(rotation=45)

### Preprocessing of text data

https://www.kaggle.com/sudalairajkumar/getting-started-with-text-preprocessing
1. Changing text to lower case
2. Removal of stopwords, punctuation and html tags from data
3. Stemming
4. Lemmatization

In [ ]:
train['low_text'] =  train['text'].str.lower()
test['low_text'] = test['text'].str.lower()

In [ ]:
string.punctuation

In [ ]:
#https://stackoverflow.com/questions/265960/best-way-to-strip-punctuation-from-a-string
train['aftr_punct'] = train['low_text'].str.translate(str.maketrans("","",string.punctuation))
test['aftr_punct'] = test['low_text'].str.translate(str.maketrans("","",string.punctuation))

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns',None)

In [ ]:
stopword = stopwords.words('english')
stopword

In [ ]:
stopword = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 
            'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 
            'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 
            'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 
            'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if',
            'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 
            'into','through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on',
            'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why',
            'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'only', 'own',
            'same', 'so', 'than', 'too', 's', 't', 'can', 'will', 'just', 'should', "should've", 'now', 'd', 'll', 'm',
            'o', 're', 've', 'y', 'ma',]

In [ ]:
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in stopword])

train['stp_text'] = train['aftr_punct'].apply(lambda text: remove_stopwords(text))
test['stp_text'] = test['aftr_punct'].apply(lambda text: remove_stopwords(text))

In [ ]:
stemmer = PorterStemmer()
def stem_word(text):
    """custom function to remove the stopwords"""
    return " ".join([stemmer.stem(word) for word in text.split()])

train['stem_text'] = train['stp_text'].apply(lambda text: stem_word(text))
test['stem_text'] = test['stp_text'].apply(lambda text: stem_word(text))

In [ ]:
lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

train['lemmatize_text'] = train['stp_text'].apply(lambda text: lemmatize_words(text))
test['lemmatize_text'] = test['stp_text'].apply(lambda text: lemmatize_words(text))

In [ ]:
train_df = train[['id','keyword','location','lemmatize_text','target']]
train_df.head()

In [ ]:
test_df = test[['id','keyword','location','lemmatize_text']]
test_df.head()

#### WordCloud

In [ ]:
plt.figure(figsize=(12,8))
wordcloud = WordCloud( background_color='white').generate(" ".join(train_df['lemmatize_text']))
plt.imshow(wordcloud)

### Converting text to vector

with test.fit_transform, we will get 11,713 features which are not same as train but with test.transform, we will get 21,032 features which are same as train

In [ ]:
bow = CountVectorizer()
train_vector = bow.fit_transform(train_df['lemmatize_text'])
test_vector = bow.transform(test_df['lemmatize_text'])

#### https://www.kaggle.com/parulpandey/getting-started-with-nlp-a-general-intro

In [ ]:
clf = LogisticRegression()
score = model_selection.cross_val_score(clf, train_vector, train_df['target'], cv=5, scoring='f1')
score

In [ ]:
clf.fit(train_vector,train_df['target'])

In [ ]:
clf_nb = MultinomialNB()
score = model_selection.cross_val_score(clf_nb, train_vector, train_df['target'], cv=5, scoring='f1')
score

In [ ]:
clf_nb.fit(train_vector, train_df['target'])

Since Naive Bayes has more accuracy than Logistic Regression, we will use it to predict test dataset target value and for final submission

In [ ]:
sample_submission = pd.read_csv("/kaggle/input/nlp-getting-started/sample_submission.csv")
sample_submission["target"] = clf_nb.predict(test_vector)
sample_submission.to_csv("submission.csv", index=False)
#sample_submission.target.value_counts()

We can upload this csv and check our score on LB. 

#### End of Notebook!!